## Herso 1 sailing with the SL lighter
### Estimate the sailing duration

- upstream 'Herso 1 + SL' longer formation
- downstream 'Herso 1 + SL' wider formation

### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import itertools
# Used for mathematical functions
import math             
import logging

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None
# spatial libraries 
# import pyproj
# import shapely.geometry
# from simplekml import Kml, Style

# package(s) for data handling
import numpy as np
import matplotlib.pyplot as plt
import tqdm


import plotly.express as px
from plotly.subplots import make_subplots

# OpenTNSim
import opentnsim

# turn on debug messages if want to see what's going on in detail
# logging.basicConfig(level=logging.DEBUG)
# turn off debug messages, only show info and higher level messages
logging.basicConfig(level=logging.INFO)

### Create vessel - add VesselProperties and ConsumesEnergy mixins

In [2]:
# Make your preferred class out of available mix-ins.
TransportResource = type(
    "Vessel",
    (
        opentnsim.core.Identifiable,
        opentnsim.core.Movable,
        opentnsim.core.VesselProperties,  # needed to add vessel properties
        opentnsim.energy.ConsumesEnergy,
        opentnsim.core.ExtraMetadata,
    ),
    {},
)  # needed to calculate resistances

In [3]:
# Create a dict with all important settings

data_vessel = {
    "env": None,
    "name": 'Vessel M6',
    "route": None,
    "geometry": None,
    "v": None,  # m/s
    "V_g_ave":4.4,
    "use_V_g_profile":False,
    "type": None,
    "B": 10.5,
    "L": 155.75,
    "H_e": None, 
    "H_f": None, 
    "T": 2,
    "safety_margin": 0.3, # for tanker vessel with rocky bed the safety margin is recommended as 0.3 m
    "h_squat": False, # if consider the ship squatting while moving, set to True, otherwise set to False. Note that here we have disabled h_squat calculation since we regard the water depth h_0 is already reduced by squat effect. This applies to figures 3, 5, 7, 8 and 9.
    "payload":None,
    "vessel_type":"Dry_DH", #vessel types: "Container","Dry_SH","Dry_DH","Barge","Tanker". ("Dry_SH" means dry bulk single hull, "Dry_DH" means dry bulk double hull)    
    "P_installed": 780.0,   
    "P_tot_given": None, # kW
    "P_tot_given_profile":False,
    "bulbous_bow": False, # if a vessel has no bulbous_bow, set to False; otherwise set to True.
    "sailing_on_power": False,
    "sailing_upstream":False,
    "wind_influence": False,
    "P_hotel_perc": 0,
    "P_hotel": None, # None: calculate P_hotel from percentage
    "x": 1,# number of propellers
    "L_w": 2.0 ,
    "C_B":0.85, 
    "C_year": 2000, # although the construction year of Herso 1 is 1961, checking with the SFC used in the paper,it is similar as year of 2000
}             



In [4]:
data_vessel_i = data_vessel.copy()
vessel = TransportResource(**data_vessel_i)

### Set vessel properties and sailing conditions ( input value )

In [5]:
# V_s = np.linspace(0, 7, 801)          # ship sailing speeds to water, (m/s)
# V_s_km_h =  np.linspace(3,20,35)

h_0 = [7.5, 4.18, 4.18, 7.83, 7.5, 6.5, 10.6, 6.45, 8.04, 5.65, 8.87, 4.93, 7.7, 9.8, 5.45]                 # water depths,(m)
power_applied_up = [620]  # canal 250kW

### prepare input matrix for calculation

In [6]:
# prepare the work to be done
# create a list of all combinations
work = list(itertools.product(h_0, power_applied_up))

# prepare a list of dictionaries for pandas
rows = []
for item in work:
    row = {"h_0": item[0], "power_applied_up": item[1]}
    rows.append(row)

# these are all the simulations that we want to run
# convert them to dataframe, so that we can apply a function and monitor progress
work_df = pd.DataFrame(rows)
work_df.iloc[13,1] = 350
work_df.tail()

,h_0,power_applied_up
10,8.87,620
11,4.93,620
12,7.70,620
13,9.80,350
14,5.45,620


### Use power2v function from OpenTNSim to get the velocity to the water

In [7]:

results = []

for i, row in tqdm.tqdm(work_df.iterrows()):
    # create a new vessel, like the one above (so that it also has L)
    
    data_vessel_i = data_vessel.copy()
    
    vessel = TransportResource(**data_vessel_i)
    vessel.P_tot_given = row['power_applied_up']
    

    # calculate the velocity that belongs to the T_strategy (while leaving the margin)
    V_w = opentnsim.strategy.power2v(vessel, h_0=row['h_0'],power_applied=row['power_applied_up'], upperbound=7)
    # V_w_down = opentnsim.strategy.power2v(vessel, h_0=row['depth'],power_applied=row['power_applied_down'], upperbound=5)
    
    result ={}
    result.update(row)
    
    result['V_w_up (m/s)'] = V_w    
    result['V_w_up (km/h)'] = V_w * 3.6
    # result_up['V_w_down (m/s)'] = V_w_down    
    # result_up['V_w_down (km/h)'] = V_w_down * 3.6
    results.append(result)


15it [00:00, 468.87it/s]


In [8]:
pv_df = pd.DataFrame(results)

pv_df.head(50)

,h_0,power_applied_up,V_w_up (m/s),V_w_up (km/h)
0,7.50,620.0,4.133651,14.881145
1,4.18,620.0,3.876978,13.957119
2,4.18,620.0,3.876978,13.957119
3,7.83,620.0,4.137343,14.894434
4,7.50,620.0,4.133651,14.881145
5,6.50,620.0,4.031934,14.514963
6,10.60,620.0,4.166338,14.998817
7,6.45,620.0,4.028326,14.501974
8,8.04,620.0,4.152477,14.948916
9,5.65,620.0,4.105619,14.780227


In [9]:
current_speeds = pd.DataFrame([-2.24,-3.4,-4.93,-2.59,-3.26,-3.85,-1.87,-1.08,-2.83,-4.99,-2.89,-6.4,-3.42,-1.64,-4.1])
current_speeds.columns =['current_speeds (km/h)']

stretch_length = pd.DataFrame([48.28,46.02,32.23,18.87,27.4,19.2,37,15,94,30,87,48,21,37.56,171])
stretch_length.columns =['stretch_length (km)']

V_g_downstream = pd.DataFrame([16,16,16,16,16,16,16,16,16,16,16,16,16,12,16])
V_g_downstream.columns =['V_g_downstream (km/h)']

V_df = pd.concat([stretch_length,current_speeds,pv_df,V_g_downstream], axis = 1)
V_df['V_g_upstream (km/h)']= V_df['V_w_up (km/h)']+V_df['current_speeds (km/h)']
V_df['V_w_down (km/h)'] = V_df['V_g_downstream (km/h)']+V_df['current_speeds (km/h)']
V_df['V_w_down (m/s)'] = V_df['V_w_down (km/h)']/3.6
V_df['duration_upstream (h)']= V_df['stretch_length (km)']/V_df['V_g_upstream (km/h)']
V_df['duration_downstream (h)']= V_df['stretch_length (km)']/V_df['V_g_downstream (km/h)']

order = ['stretch_length (km)','h_0','current_speeds (km/h)','power_applied_up',
         'V_w_up (m/s)','V_g_upstream (km/h)','duration_upstream (h)',
         'V_w_down (m/s)','V_g_downstream (km/h)','duration_downstream (h)']
V_df = V_df[order]

### display the sailing speed unstream and down stream per stretch

In [10]:
V_df

,stretch_length (km),h_0,current_speeds (km/h),power_applied_up,V_w_up (m/s),V_g_upstream (km/h),duration_upstream (h),V_w_down (m/s),V_g_downstream (km/h),duration_downstream (h)
0,48.28,7.50,-2.24,620.0,4.133651,12.641145,3.819274,3.822222,16,3.017500
1,46.02,4.18,-3.40,620.0,3.876978,10.557119,4.359144,3.500000,16,2.876250
2,32.23,4.18,-4.93,620.0,3.876978,9.027119,3.570353,3.075000,16,2.014375
3,18.87,7.83,-2.59,620.0,4.137343,12.304434,1.533594,3.725000,16,1.179375
4,27.40,7.50,-3.26,620.0,4.133651,11.621145,2.357771,3.538889,16,1.712500
5,19.20,6.50,-3.85,620.0,4.031934,10.664963,1.800288,3.375000,16,1.200000
6,37.00,10.60,-1.87,620.0,4.166338,13.128817,2.818228,3.925000,16,2.312500
7,15.00,6.45,-1.08,620.0,4.028326,13.421974,1.117570,4.144444,16,0.937500
8,94.00,8.04,-2.83,620.0,4.152477,12.118916,7.756469,3.658333,16,5.875000
9,30.00,5.65,-4.99,620.0,4.105619,9.790227,3.064280,3.058333,16,1.875000


### get the sailing duration (hours) of unstream trip, downstream trip and round trip

In [11]:
upstream_total_duration = V_df['duration_upstream (h)'].sum()
upstream_total_duration

66.9013008868138

In [12]:
(upstream_total_duration-64.19)/64.19 # difference compared with the results from paper (Schweighofer & Suvacarov, 2018)

0.04223868027440104

In [13]:
downstream_total_duration = V_df['duration_downstream (h)'].sum()
downstream_total_duration

46.5675

In [14]:
(downstream_total_duration-46.56)/46.56 # difference compared with the results from paper (Schweighofer & Suvacarov, 2018)

0.00016108247422681022

In [15]:
roundtrip_duration = upstream_total_duration + downstream_total_duration
roundtrip_duration

113.4688008868138

In [16]:
(roundtrip_duration-110.94)/110.94 # difference compared with the results from paper (Schweighofer & Suvacarov, 2018)

0.02279431122060391